# CNN - Problem B (PoseNet)

In [1]:
# General requirements
import os
import pandas as pd
import numpy as np

# MLflow dashboard
import mlflow
mlflow.set_tracking_uri('http://35.228.45.76:5000')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='../../keys/mlflow-312506-8cfad529f4fd.json'

# Import data augmentation
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from augmentation.methods import *

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Set random state
import numpy as np
random_state = 47
np.random.seed(random_state)

# Use GPU (if available)
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices( 'GPU' )
print( 'Num GPUs Available: ', len( physical_devices ) )
if len( physical_devices ) > 0:  
    tf.config.experimental.set_memory_growth( physical_devices[0], True )

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Num GPUs Available:  0


# 1. Data Preparation

## 1.1 Load Dataset

In [2]:
posenet_dataset_path = '../../datasets/posenet-uncut/'
kinect_dataset_path = '../../datasets/kinect_fixed_not_cut/'
cut_dataset_path = '../../datasets/new_posenet_marked_start_end/'
goodness_score = pd.read_csv('../../datasets/VideoScoring.csv')
exrecise_score = pd.read_csv('../../datasets/ExerciseScoring.csv')

train_test_ratio = 0.9
new_label = 'ExreciseScore'
exersice_score_indicator = 'O_Score'
goodness_score_indicator = 'AVG'
goodness_score_threshold = 3

In [3]:
# Drop video augmentation results
goodness_score = goodness_score[goodness_score['FileName'].str.match('U')==False]
goodness_score = goodness_score[goodness_score[goodness_score_indicator]>=goodness_score_threshold]

posenet_ok = goodness_score['FileName']+'.csv'

exrecise_score['Posenet'] = exrecise_score['Posenet'] + '.csv'
exrecise_score['Kinect'] = exrecise_score['Kinect'] + '.csv'

In [4]:
posenet_files = []
kinect_files = []

for file in os.listdir(posenet_dataset_path):
    if not file.find(".csv",0) == -1:
        if file in (goodness_score['FileName']+'.csv').to_list():
            posenet_files.append(file)
        
for file in os.listdir(kinect_dataset_path):
    if not file.find(".csv",0) == -1:
        if file in (goodness_score['FileName']+'_kinect.csv').to_list():
            kinect_files.append(file)
        
print('Total posenet datasets: {}'.format(len(posenet_files)))
print('Total kinect datasets: {}'.format(len(kinect_files)))

Total posenet datasets: 157
Total kinect datasets: 157


In [5]:
TRAIN_SPLIT_POSENET = int(len(posenet_files)*train_test_ratio)
TRAIN_SPLIT_KINECT = int(len(kinect_files)*train_test_ratio)
print(TRAIN_SPLIT_POSENET,TRAIN_SPLIT_KINECT)

141 141


In [6]:
def data_augmentation(df):
    augmented_datasets = []
    
    # Add original data
    augmented_datasets.append(df)
    
    # Mirror X coordinate
    for i in range(len(augmented_datasets)):
        augmented_datasets.append(mirror(augmented_datasets[i],'x', append=False))

    # Stretch by 50%
    for i in range(len(augmented_datasets)):
        df_temp = augMultiplier(augmented_datasets[i].drop(columns=[new_label]), multiplier=1.5)
        df_temp[new_label] = augmented_datasets[i][new_label]
        augmented_datasets.append(df_temp)
    

    # Compress by 25%
    for i in range(len(augmented_datasets)):
        df_temp = augMultiplier(augmented_datasets[i].drop(columns=[new_label]), multiplier=0.25)
        df_temp[new_label] = augmented_datasets[i][new_label]
        augmented_datasets.append(df_temp)
   

    # Rotate by p/7
#     samples = df.sample(5000)
#     angle = 3.1415 / 7
#     samples_rotated = rotate(samples.drop(columns=[new_label]), angle=angle, posenet=False)
#     samples_rotated[new_label] = samples[new_label].append(samples[new_label], ignore_index=True)
#     df = df.append(samples_rotated, ignore_index=True)
#     print(df.shape)

    # Rotate by -p/9
#     samples = df.sample(5000)
#     angle = 3.1415 / -9
#     samples_rotated = rotate(samples.drop(columns=[new_label]), angle=angle, posenet=False)
#     samples_rotated[new_label] = samples[new_label].append(samples[new_label], ignore_index=True)
#     df = df.append(samples_rotated, ignore_index=True)
#     print(df.shape)

    return augmented_datasets

In [7]:
HISTORY_SIZE = 10
STEP = 20
N_FEATURES = 26

def convert_data(dataset, target_column, history_columns):

    segments = []
    labels = []

    for i in range(0,len(dataset) - HISTORY_SIZE, STEP):
        segments.append(dataset[history_columns].values[i:i + HISTORY_SIZE])
        labels.append(dataset[target_column].mean())
    
    return np.array(segments).reshape(-1,HISTORY_SIZE*N_FEATURES), np.array(labels)

In [8]:
history_columns = ['head_x', 'head_y', 'left_shoulder_x', 'left_shoulder_y',
       'right_shoulder_x', 'right_shoulder_y', 'left_elbow_x', 'left_elbow_y',
       'right_elbow_x', 'right_elbow_y', 'left_wrist_x', 'left_wrist_y',
       'right_wrist_x', 'right_wrist_y', 'left_hip_x', 'left_hip_y',
       'right_hip_x', 'right_hip_y', 'left_knee_x', 'left_knee_y',
       'right_knee_x', 'right_knee_y', 'left_ankle_x', 'left_ankle_y',
       'right_ankle_x', 'right_ankle_y']
target_column = 'ExreciseScore'

def read_dataset(path,file_list,split_point,is_posenet=True,isTrain=True):
    all_segments = None
    all_labels = None
    
    start=0
    end=0
    
    if isTrain:
        start = 0
        end = split_point
    else:
        start = split_point
        end = len(file_list)
    
    for file in file_list[start:end]:
        df = pd.read_csv(path+file)
        
        if is_posenet:  
            df = df[df.columns.drop(list(df.filter(regex='_eye_')))]
            df = df[df.columns.drop(list(df.filter(regex='_ear_')))]
            df = df[df.columns.drop(list(df.filter(regex='score')))]
            df = df.rename(columns={'nose_x': 'head_x', 'nose_y': 'head_y'})
            df[new_label] = float(exrecise_score[exrecise_score['Posenet'] == file][exersice_score_indicator])
        else:  
            df = df.drop(columns=['Unnamed: 0','FrameNo'])
            df[new_label] = float(exrecise_score[exrecise_score['Kinect'] == file][exersice_score_indicator])
            
        # Cut start / end
        try:
            se = pd.read_csv('../../datasets/new_posenet_marked_start_end/'+file)
            cut_start = min(se[(se['start']==0) * (se['end']==0)].index)
            cut_end = max(se[(se['start']==0) * (se['end']==0)].index)
            df = df.iloc[cut_start:cut_end]
        except IOError as e:
            print('Error in reading file: ', e)
         
        if all_segments is None:
            all_segments = df[history_columns]
            all_labels = df[target_column]
        else:
            all_segments = np.append(all_segments,df[history_columns],axis=0)  
            all_labels = np.append(all_labels,df[target_column],axis=0)
            
    
        if isTrain:
            for sets in data_augmentation(df):
                all_segments = np.append(all_segments,sets[history_columns],axis=0)  
                all_labels = np.append(all_labels,sets[target_column],axis=0)  

                sys.stdout.write("Total added rows: %d   \r" % (all_segments.shape[0]) )
                sys.stdout.flush()
        
    
    return all_segments,all_labels

In [9]:
X_train, y_train = read_dataset(posenet_dataset_path,posenet_files,TRAIN_SPLIT_POSENET,True,True)
X_test, y_test = read_dataset(posenet_dataset_path,posenet_files,TRAIN_SPLIT_POSENET,True,False)

Error in reading file:  [Errno 2] No such file or directory: '../../datasets/new_posenet_marked_start_end/A60.csv'


In [10]:
X_train.shape

(640053, 26)

In [11]:
X_test.shape

(6689, 26)

In [12]:
y_train

array([0.      , 0.      , 0.      , ..., 3.948862, 3.948862, 3.948862])

## 1.3 Standardize features and labels

In [13]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1,1))

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape)

Training features shape: (640053, 26)
Training labels shape: (640053, 1) 

Test features shape: (6689, 26)
Test labels shape: (6689,)


## 1.4 Reshape data for convolutional layers

In [17]:
C = 1

original_shape = X_train
X_train = X_train.reshape((original_shape.shape[0], C, original_shape.shape[1]))
X_test = X_test.reshape((X_test.shape[0], C, X_test.shape[1]))
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

print("Before: {}".format(original_shape))

input_shape = (n_timesteps, n_features)

print("After: {}".format(X_train.shape))

Before: [[ 0.87950181 -0.44858478  0.87045823 ...  0.14037643  0.88156627
   0.14203014]
 [ 0.87950181 -0.44858478  0.87045823 ...  0.14037643  0.88156627
   0.14203014]
 [ 0.88387521 -0.44255569  0.8770654  ...  0.137868    0.88232331
   0.14709143]
 ...
 [-0.5234906  -0.92524202 -0.52601899 ... -0.95120237 -0.52541417
  -0.94454015]
 [-0.52298005 -0.92331228 -0.52671667 ... -0.95192255 -0.52542601
  -0.94421066]
 [-0.52298005 -0.92331228 -0.52671667 ... -0.95192255 -0.52542601
  -0.94421066]]
After: (640053, 1, 26)


# 2. Define Model Architecture

## 2.1 Model Architecture

The **create_model()** method returns a model with the layers and configurations specified.

The model will optimize the mean squared error (mse) required for regression problems.

In [18]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError
from tensorflow.keras.layers import InputLayer

metrics = [
    MeanSquaredError(name="mse", dtype=None),
    MeanAbsoluteError(name="mae", dtype=None),
    RootMeanSquaredError(name="rmse", dtype=None),
]

def create_model(layers, optimizer):
    model = tf.keras.models.Sequential(
        InputLayer(input_shape=(n_timesteps, n_features))
    )

    for layer in layers:
        model.add(layer)

    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=metrics)

    print(model.summary())

    return model

# 3. Run Experiments

We will use our [MLflow dashboard](http://35.228.45.76:5000/#/) to track the outcome of experimentation runs.

## 3.1 Define a MLflow Experiment

We'll define the running setup and the model signature.

In [19]:
#verbose, epochs, batch_size = 1, 150, 256
#verbose, epochs, batch_size = 1, 150, 128
#verbose, epochs, batch_size = 1, 150, 64
verbose, epochs, batch_size = 1, 150, 32
#verbose, epochs, batch_size = 1, 150, 16

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

In [20]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

input_schema = Schema([
    ColSpec("double", "head_x"),
    ColSpec("double", "head_y"),
    ColSpec("double", "left_shoulder_x"),
    ColSpec("double", "left_shoulder_y"),
    ColSpec("double", "right_shoulder_x"),
    ColSpec("double", "right_shoulder_y"),
    ColSpec("double", "left_elbow_x"),
    ColSpec("double", "left_elbow_y"),
    ColSpec("double", "right_elbow_x"),
    ColSpec("double", "right_elbow_y"),
    ColSpec("double", "left_wrist_x"),
    ColSpec("double", "left_wrist_y"),
    ColSpec("double", "right_wrist_x"),
    ColSpec("double", "right_wrist_y"),
    ColSpec("double", "left_hip_x"),
    ColSpec("double", "left_hip_y"),
    ColSpec("double", "right_hip_x"),
    ColSpec("double", "right_hip_y"),
    ColSpec("double", "left_knee_x"),
    ColSpec("double", "left_knee_y"),
    ColSpec("double", "right_knee_x"),
    ColSpec("double", "right_knee_y"),
    ColSpec("double", "left_ankle_x"),
    ColSpec("double", "left_ankle_y"),
    ColSpec("double", "right_ankle_x"),
    ColSpec("double", "right_ankle_y"),
])
output_schema = Schema([
    ColSpec("double", "ExreciseScore")
])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

## 3.2 Plot Training History

This method will be used to inspect how the model is learning by showing training and validation loss.

In [21]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

## 3.3 Define Evaluation Metrics

We'll use standard regression performance metrics to evaluate model performance.

In [22]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error

def eval_metrics(actual, pred):
    mse = mean_squared_error(actual, pred)
    msa = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    variance = explained_variance_score(actual, pred)
    return mse, msa, r2, variance

## 3.4 Model Parameters

For each experimentation run, we'll test different values for model parameters with each model architecture we define. These are the parameter values we're using as a starting point:

- We'll begin small kernels, and then increase to larger ones. 
- As for layers, we'll start by using the ReLU activation function and He weight initialization, which we have noticed seem to provide a general good model performance.
- The RMSprop optimizer will be used first as this is a regression problem we're trying to solve, and we'll use a modest learning rate of 0.001 with a large momentum of 0.9 (generally good practices). But we'll also try the Adam optimizer.

As we're experimenting with these values, the results will be shown in [MLflow dashboard](http://35.228.45.76:5000/#/).

In [23]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

#filters = 16
filters = 32

kernel_size = 3
#kernel_size = 5

activation = 'relu'
kernel_initializer = 'he_uniform'

pool_size = 2

dense_units = 32
#dense_units = 64

output_activation = ''
#output_activation = 'linear'

# Optimization function
optimizer = 'Adam'
learning_rate = 0.01
#opt = SGD(learning_rate=learning_rate, momentum=0.9)
opt = Adam(learning_rate=learning_rate)
#opt = RMSprop(learning_rate=learning_rate, momentum=0.9)

## 3.5 Model

Now we can start experimenting with the models, and we'll start with a baseline model. The baseline model will establish a minimum model performance to which all other models can be compared. 

For the baseline model, we've been inspired by the general architectural principles of VGG models - but with 1 dimensional convolution. 
This seems like a good starting point because that structure is easy to understand and implement at the same time as it has previously achieved good performance. <br />
The architecture involves stacking convolutional layers with small 3x3 filters followed by a max-pooling layer. Padding is used on the convolutional layers to ensure the height and width of the output feature maps matches the inputs.

The feature maps output from the feature extraction part of this baseline model will be flattened before we can interpret them with two fully connected (Dense) layers, and then output a prediction.

This baseline will also be tested with larger filters as we're experimenting with parameter values.

In [ ]:
import time
from tensorflow.keras.layers import Conv1D, Dropout, MaxPool1D, Flatten, Dense

model_name = 'scoring_cnn_posenet'

layers = [
    Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation=activation, kernel_initializer=kernel_initializer),
    MaxPool1D(pool_size=pool_size, padding='same'),
    Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation=activation, kernel_initializer=kernel_initializer),
    MaxPool1D(pool_size=pool_size, padding='same'),
    #Dropout(0.2),
    Flatten(), 
    Dense(dense_units, activation=activation, kernel_initializer=kernel_initializer),
    Dense(n_outputs)
    #Dense(n_outputs, activation=output_activation)
]

with mlflow.start_run(run_name=model_name) as run:

    model = create_model(layers=layers, optimizer=opt)

    history = model.fit(X_train, 
                        y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_split=0.2, 
                        shuffle=True, 
                        verbose=verbose, 
                        callbacks=[early_stopping])

    # Plot training history
    plot_train_history(history, 'Training and validation loss')
    plt.savefig("training_history.jpg")
    mlflow.log_artifact("training_history.jpg")
    plt.show()

    # Log model parameters
    mlflow.log_param("activation", activation)
    mlflow.log_param("kernel_initializer", kernel_initializer)
    mlflow.log_param("output activation", output_activation)
    mlflow.log_param("optimizer", optimizer)
    mlflow.log_param("learning rate", learning_rate)
    mlflow.log_param("batch size", batch_size)
    mlflow.log_param("epochs", early_stopping.stopped_epoch)
    mlflow.log_param("filters", filters)
    mlflow.log_param("kernel_size", kernel_size)
    mlflow.log_param("pool size", pool_size)
    mlflow.log_param("total params", model.count_params())
    mlflow.log_param("units", dense_units)

    # Log model prediction time
    predictions = None
    times = []
    for i in range(10):
        start_time = time.time()
        predictions = model.predict(X_test)
        end_time = time.time()
        process_time = (end_time - start_time) * 1000
        times.append(process_time)
    process_time = sum(times) / len(times)

    # Log model performance
    predictions = y_scaler.inverse_transform(predictions)
    (mse, mae, r2, variance) = eval_metrics(y_test, predictions)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("R-squared", r2)
    mlflow.log_metric("variance", variance)
    mlflow.log_metric("process time", process_time)

    # Log model and scaler(s)
    mlflow.keras.log_model(model, model_name, signature=signature)
    mlflow.sklearn.log_model(X_scaler, 'InputScaler')
    mlflow.sklearn.log_model(y_scaler, 'OutputScaler')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1, 32)             2528      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 32)             3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 32)             0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

In [ ]:
predictions = model.predict(X_test, verbose=1)
# Invert transform on predictions
predictions = y_scaler.inverse_transform(predictions)

In [ ]:
(mse, msa, r2, variance) = eval_metrics(y_test, predictions)

print('MSE: ', mse)
print('MSA: ', msa)
print('R-Squared: ', r2)
print('Explained Variance Score: ', variance)

## Discussion

- No difference between optimizers when using large batch sizes (except SGD performed slightly worse). When using the largest batch size, the model always stopped training early (epochs = 100).

- No activation function in output layer.

- Small batch sizes did not give any good performance.